# IS434: Social Analytics & Apps
### SkillsFuture Industry Scraper
---

In [1]:
import pandas as pd
import os
from os.path import isfile, join
import aspose.words as aw
pd.set_option('display.max_rows', None)

Exception ignored in: 'FieldInitializationWarning'
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
ImportError: the static field 'transparent' of type 'aspose.pydrawing.Brushes' is not initialized
Exception ignored in: 'FieldInitializationWarning'
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
ImportError: the static field 'alice_blue' of type 'aspose.pydrawing.Brushes' is not initialized
Exception ignored in: 'FieldInitializationWarning'
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
ImportError: the static field 'antique_white' of type 'aspose.pydrawing.Brushes' is not initialized


## Clean file names, convert to PDF to HTML

In [2]:
# fix file names after importing - to be run once only after scraping

# def clean_title( file_name ):
#     temp = file_name.replace("Skills Map_", "")
#     temp = temp.replace("(GSC Top 5)", "")
#     temp = temp.split(" ")
#     temp = "-".join(temp)
#     temp = temp.replace("-.pdf", ".pdf")
#     new_name = temp.lower()
#     return new_name
    
# # Retrieve PDF files
# pdf_files = [file for file in os.listdir("./pdf") if file.endswith('.pdf')]

# for file in pdf_files:
#     # Clean file title and rename file
#     new_name = clean_title( file )
#     os.rename(f"./pdf/{file}", f"./pdf/{new_name}")
    
#     # Save PDF as HTML document
#     doc = aw.Document(f"./pdf/{new_name}")
#     doc.save(f"./html/{new_name[:-4]}.html")

# # Remove images generated from the package...
# for file in os.listdir('./html/'):
#     if file.endswith('.png'):
#         os.remove(f'./html/{file}')

# print('Done!')

### Scrape the HTML for skills

In [3]:
# Helper function to extract job role
def retrieve_job_role( span_list ):
    for i in range( len(span_list) ):
        span_text = span_list[i].text.strip()

        if "Job Role" == span_text:
            return span_list[i+1].text

In [6]:
# Helper function to extract skills
def retrieve_skills( span_list ):
    end_index = 0
    start_index = 0
    
    # Get index of last occurrence of skill
    for i in range( len(span_list)-1, 0, -1 ):
        text = span_list[i].text
        if "Level" in text:
            end_index = i
            break

    # Get index of first occurrence of skill
    for i in range( len(span_list)-1 ):
        text = span_list[i].text
        if "Generic Skills and Competencies" in text:
            start_index = i+1
            break
    
    # Get the subset of <span> that have the skills
    skillsets = [ span.text for span in span_list[start_index:end_index+1] ]

    # Remove '\xa0' occurrences in list from pdf to html conversion
    skillsets = [ skill for skill in skillsets if skill.strip() not in "\xa0"]
    
    # Store as list of skills and return
    skills_list = []
    
    for i in range(1, len(skillsets), 2):
        skill_name = skillsets[i-1].strip()
        skill_level = skillsets[i].strip()

        skills_list.append(f"{skill_name} {skill_level}")
    
    return skills_list

In [7]:
from bs4 import BeautifulSoup

jobs_dict = {}

# Retrieve html files from the directory
html_files = [file for file in os.listdir("./html") if file.endswith('.html')]

# Scrape each file
for file in html_files:
    with open(f"./html/{file}") as fp:
        soup = BeautifulSoup(fp, 'html.parser')
        all_spans = soup.find_all('span')
        
        # Retrieve job role
        job_role = retrieve_job_role( all_spans )

        # Retrieve job's skills and competencies
        skills_list = retrieve_skills( all_spans )
        
        # Create key-value pair -> job_role, skill_list
        jobs_dict[ job_role ] = skills_list
        
        
# Remember to convert Beginner = Level 1, Intermediate = Level 2, Advanced = Level 3

In [10]:
jobs_dict

{'Artificial Intelligence Applied Researcher ': ['Business Innovation Level 5',
  'Leadership Advanced',
  'Business Needs Analysis Level 5',
  'Developing People Advanced',
  'Computer Vision Technology Level 4',
  'Communication Intermediate',
  'Data Design Level 5',
  'Computational Thinking Intermediate',
  'Data Ethics Level 5',
  'Lifelong Learning Intermediate',
  'Data Governance Level 5',
  'Data Strategy Level 5',
  'Design Thinking Practice Level 5',
  'Emerging Technology Synthesis Level 5',
  'Intelligent Reasoning Level 5',
  'Organisational Design Level 5',
  'Pattern Recognition Systems Level 5',
  'Project Management Level 5',
  'Quality Standards Level 5',
  'Research Level 4',
  'Self-learning Systems Level 4',
  'Stakeholder Management Level 4',
  'Text Analytics and Processing Level 6'],
 'Artificial Intelligence/Machine Learning Engineer ': ['Business Needs Analysis Level 4',
  'Leadership Intermediate',
  'Cloud Computing Level 3',
  'Developing People Intermedi

In [ ]:
# DO BE DONE
#         # Retrieve job's key task
#         all_td = soup.find_all('td')
#         start_index = 0
#         end_index = 0

#         # Find range of <td> tags that contain job description information
#         for i in range( len(all_td) ):
#             row_td = all_td[i]

# #             if "Performance Expectations" in row_td.p:
# #                 print(row_td.p)
#             print( row_td.p.string )

# #             for span in p_tag:
# #                 span_text = span.text.strip()
# #                 if "Performance Expectations" == span_text:
# #                     start_index = i + 1

# #                 if "Skills and Competencies" == span_text:
# #                     end_index = i

#         # Extract <p> tags that contain job description
#         job_desc_list = all_p[start_index:end_index]

#         print( job_desc_list )